Web scraping

https://www.thepythoncode.com/article/download-web-page-images-python

In [1]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse
import re
import torch
import cv2
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    if "preview" not in url:
      return False
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
soup = bs(requests.get("https://patternbank.com/studio?image_format=vector&in_repeat=1&licence=stock&page=1").content, "html.parser")

In [5]:
def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    labels=[]
    for img in soup.findAll("img",{"class":"img-responsive"}):
        img_url = img.attrs.get("data-src")
        
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
            # make the URL absolute by joining domain with the URL that is just extracted
        # print(img_url)
        img_url = urljoin(url, img_url) 
        
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
            # print(img_url)
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            # print("is valid\n")
    
            page_url=urljoin(url,img.parent["href"])
            
            name=f"{page_url.rsplit('/', 1)[-1]}.jpg"
            soup_page = bs(requests.get(page_url).content, "html.parser")
            categories=[]
            all_category_spans=soup_page.findAll('b', text=re.compile('Categories:'))
            if len(all_category_spans)==0:
              continue
            for span in all_category_spans[0].parent.findAll("span"):
              categories.append(span.text)
            
            urls.append([img_url,name,categories])
            # labels.append(categories)
        # else:
        #   print("is not valid\n")
    return urls

In [6]:
# test_urls=get_all_images("https://patternbank.com/studio?image_format=vector&in_repeat=1&licence=stock&page=1")

In [7]:
# def get_all_images(url):
#     """
#     Returns all image URLs on a single `url`
#     """
#     soup = bs(requests.get(url).content, "html.parser")
#     urls = []
#     labels=[]
#     for img in soup.find("div",{"class":"row design-thumbnails product-image-grid js-image-grid"}).findAll("img"):
#         img_url = img.attrs.get("src")
        
#         if not img_url:
#             # if img does not contain src attribute, just skip
#             continue
#             # make the URL absolute by joining domain with the URL that is just extracted
#         # print(img_url)
#         img_url = urljoin(url, img_url) 
        
#         try:
#             pos = img_url.index("?")
#             img_url = img_url[:pos]
#             # print(img_url)
#         except ValueError:
#             pass
#         # finally, if the url is valid
#         if is_valid(img_url):
#             # print("is valid\n")
#             print(img_url)
#             print()
#             page_url=urljoin(url,img.parent["href"])
            
#             name=f"{page_url.rsplit('/', 1)[-1]}.jpg"
#             soup_page = bs(requests.get(page_url).content, "html.parser")
#             categories=[]
#             all_category_spans=soup_page.findAll('b', text=re.compile('Categories:'))
#             if len(all_category_spans)==0:
#               continue
#             for span in all_category_spans[0].parent.findAll("span"):
#               categories.append(span.text)
            
#             urls.append([img_url,name,categories])
#             # labels.append(categories)
#         # else:
#         #   print("is not valid\n")
#     return urls

In [8]:
def download(url, pathname,name):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # print(url)
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    # filename = os.path.join(pathname, url.split("/")[-1])
    filename = os.path.join(pathname,name)
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    # progress = tqdm(, f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in response.iter_content(1024):
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            # progress.update(len(data))

In [9]:
def download_all(imgs, path):
    for image_url,name,labels in tqdm(imgs):
        # for each image, download it
        download(image_url,path,name)
    

In [10]:
# len(test_urls)

In [11]:
# download_all(test_urls,"/content/drive/MyDrive/fabrics")

In [12]:
import pandas as pd

In [13]:
NUM_PAGES=419
imgs=[]
# get all images
for i in tqdm(range(1,NUM_PAGES)):
    url=f"https://patternbank.com/search?button=&licence=stock&page={i}&q=flower&search_type=designs&sort=best-match&utf8=%E2%9C%93"
    imgs.extend(get_all_images(url))

  0%|                                                                                           | 0/418 [00:00<?, ?it/s]/tmp/ipykernel_18312/3167404411.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_category_spans=soup_page.findAll('b', text=re.compile('Categories:'))
100%|███████████████████████████████████████████████████████████████████████████████| 418/418 [9:11:42<00:00, 79.19s/it]


In [14]:
len(imgs)

41723

In [15]:
import pickle

In [16]:
f = open('floral.pickle', 'wb')
pickle.dump(imgs, f)

In [30]:
f2 = open('floral.pickle', 'rb')
imgs1 = pickle.load(f2)

In [31]:
len(imgs1)

41723

In [ ]:
imgs=list(dict.fromkeys(imgs))

In [ ]:

with open(r'urls.txt', 'w') as fp:
    for item in imgs:
        fp.write("%s\n" % item)
print('Done')

In [ ]:
# download and save all images
download_all(imgs1,"Floral")

  7%|█████▎                                                                      | 2898/41723 [11:58<3:47:12,  2.85it/s]

In [19]:
import pandas as pd

In [20]:
selection1=pd.read_csv("selection1_meta_data.csv")

In [21]:
path="Floral"

In [22]:
df=pd.DataFrame(imgs1,columns=["url","name","labels"])


In [23]:
intersect=df[df.name.isin(list(set(df["name"]).intersection(set(selection1["name"]))))].reset_index(drop=True)

In [27]:
for filename in intersect.name:
    try:
        os.remove(join(path,filename))
    except:
        pass

In [28]:
diff=df[df.name.isin(list(set(df["name"]).difference(set(selection1["name"]))))].reset_index(drop=True)

In [29]:
diff.to_csv("floral_diff.csv")

In [ ]:
label_dummies=df["labels"].apply(lambda x: " ".join(x)).str.get_dummies(sep=" ")
label_dummies=label_dummies.drop("Skins",axis=1).rename(columns={"Animal": "Animal_Skins"})
meta=pd.concat([df, label_dummies], axis='columns')
label_dummies

,Abstract,Animal_Skins,Animals/Birds,Border,Camouflage,Checks,Conversationals,Ethnic,Floral,Geometric,Nature,Paisleys,Placements,Stripes,Texture,Traditional,Tribal,Tropical
0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9462,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0
9463,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0
9464,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
9465,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [ ]:
meta.to_csv("stripes.csv")

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, csv, train, test):
        self.csv = csv
        self.train = train
        self.test = test
        self.all_image_names = self.csv[:]['Id']
        self.all_labels = np.array(self.csv.drop(['Id', 'Genre'], axis=1))
        self.train_ratio = int(0.85 * len(self.csv))
        self.valid_ratio = len(self.csv) - self.train_ratio
        # set the training data images and labels
        if self.train == True:
            print(f"Number of training images: {self.train_ratio}")
            self.image_names = list(self.all_image_names[:self.train_ratio])
            self.labels = list(self.all_labels[:self.train_ratio])
            # define the training transforms
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((400, 400)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=45),
                transforms.ToTensor(),
            ])
        # set the validation data images and labels
        elif self.train == False and self.test == False:
            print(f"Number of validation images: {self.valid_ratio}")
            self.image_names = list(self.all_image_names[-self.valid_ratio:-10])
            self.labels = list(self.all_labels[-self.valid_ratio:])
            # define the validation transforms
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((400, 400)),
                transforms.ToTensor(),
            ])
        # set the test data images and labels, only last 10 images
        # this, we will use in a separate inference script
        elif self.test == True and self.train == False:
            self.image_names = list(self.all_image_names[-10:])
            self.labels = list(self.all_labels[-10:])
             # define the test transforms
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),
            ])
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, index):
        image = cv2.imread(f"../input/movie-classifier/Multi_Label_dataset/Images/{self.image_names[index]}.jpg")
        # convert the image from BGR to RGB color format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # apply image transforms
        image = self.transform(image)
        targets = self.labels[index]
        
        return {
            'image': torch.tensor(image, dtype=torch.float32),
            'label': torch.tensor(targets, dtype=torch.float32)
        }

In [25]:
from os import listdir
from os.path import join
from zipfile import ZipFile
from zipfile import ZIP_DEFLATED
 

In [38]:
files = [join(path,f) for f in listdir(path)]
    

In [39]:
files[5]

'ditsy/560185571-ditsy.jpg'

In [27]:
# len(files)

In [56]:
with ZipFile(f'stripes.zip', 'w', compression=ZIP_DEFLATED) as handle:
  # add all files to the zip
  for filepath in tqdm(files):
      # add file to the archive
      handle.write(filepath)

100%|██████████████████████████████████████████████████████████████████████████████| 8324/8324 [00:25<00:00, 321.04it/s]


In [57]:
!pwd

/bi/ala/scratch/pmehrbod/GAN
